In [1]:
!pip install trl

In [2]:
import re
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig
from trl import GRPOConfig, GRPOTrainer
import os
os.environ["WANDB_MODE"] = "disabled"

model_name = "Qwen/Qwen3-0.6B"
#model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"


2025-05-20 07:38:04.561357: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747726684.583987     132 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747726684.590909     132 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Load and prep dataset

SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

In [4]:
XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

In [5]:
def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

In [6]:
def extract_hash_answer(text: str) -> str:  ## | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip().replace(",", "").replace("$", "")

In [7]:
# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            #{'role': 'user', 'content': 'What is the largest single-digit prime number?'},
            #{'role': 'assistant', 'content': XML_COT_FORMAT.format(
            #    reasoning="9 is divisble by 3 and 8 is divisible by 2, but 7 is prime.",
            #    answer="7"
            #)},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) 
    return data 

In [8]:
dataset = get_gsm8k_questions()
dataset = dataset.select(range(50))

In [9]:
def debug_wrapper(func):
    def inner(*args, **kwargs):
        rewards = func(*args, **kwargs)
        print(f"{func.__name__}: {rewards}")
        return rewards
    return inner

In [10]:
# Reward based on exact correctness of the answer extracted from XML
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    # Reward 2.0 if the extracted response exactly matches the gold answer, otherwise 0.0
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

In [11]:
# Reward if the extracted response is an integer (useful for math problems)
def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    # Reward 0.5 if the output is a digit, otherwise 0.0
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

In [12]:
# Reward only if the output matches a strict XML format with newlines and exact tag structure
def strict_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    # Reward 0.5 if format exactly matches strict XML structure, else 0.0
    return [0.5 if match else 0.0 for match in matches]

In [13]:
# Reward more leniently if the XML tags are present, regardless of newline structure
def soft_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    # Reward 0.5 if XML tags are present in any format, else 0.0
    return [0.5 if match else 0.0 for match in matches]

In [14]:
# Helper function: scores how well the text follows the expected XML tag structure
def count_xml(text) -> float:
    count = 0.0
    # Each correctly placed XML tag adds 0.125
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        # Penalize any trailing content after </answer>
        count -= len(text.split("\n</answer>\n")[-1]) * 0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1) * 0.001
    return count

# Reward function using the custom XML structure scoring above
def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    # Reward based on partial correctness of XML formatting
    return [count_xml(c) for c in contents]

In [15]:
if "deepseek" in model_name:
    output_dir = "outputs/DeepSeek-R1-Distill-Qwen-1.5B"
    run_name = "DeepSeek-R1-Distill-Qwen-1.5B"
else:
    output_dir="outputs/Qwen3-0.6B-GRPO"
    run_name="Qwen3-0.6B-GRPO-gsm8k"

In [16]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    ## attn_implementation="flash_attention_2",
    device_map=None
).to("cuda")

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [17]:
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        print(name)

model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.o_proj
model.layers.2.mlp.gate_proj
model.layers.2.mlp.up_proj
model.layers.2.mlp.down_proj
model.layers.3.self_attn.q_proj
model.layers.3.self_attn.k_proj
model.layers.3.self_attn.v_proj
model.layers.3.self_attn.o_proj
model.layers.3.mlp.gate_proj
model.layers.3.mlp.up_proj
model.layers.3.mlp.down_proj
model.layers.4.self_attn.q_proj
model.layers.4.self_attn.k_proj
model.layers.4.self_attn.v_proj
model.layers.4.self_attn.o_proj
model.layers.4.mlp.g

In [34]:
training_args = GRPOConfig(
    output_dir=output_dir,
    run_name=run_name,
    learning_rate=5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type='cosine',
    logging_steps=1,
    bf16=False,  
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_generations=4,
    max_prompt_length=256,
    max_completion_length=786,
    num_train_epochs=1,
    save_steps=100,
    max_grad_norm=1.0,
    # report_to="wandb",
    log_on_each_node=False,
)

In [35]:
peft_config = LoraConfig(
    r=16,                             # Rank of the LoRA decomposition (low-rank matrix size)
    lora_alpha=64,                   # Scaling factor for the LoRA update
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj", "gate_proj"],            # Which modules to apply LoRA to
    task_type="CAUSAL_LM",           # Tells PEFT it's for autoregressive language modeling
    lora_dropout=0.05,               # Dropout applied to LoRA layers during training
)

In [36]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [37]:
def easy_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]["content"].lower() for completion in completions]
    rewards = []
    for r in responses:
        if "<answer>" in r and "</answer>" in r:
            rewards.append(1.0)
        elif "<answer>" in r or "</answer>" in r:
            rewards.append(0.5)
        else:
            rewards.append(0.0)
    return rewards


In [38]:
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        debug_wrapper(easy_reward_func)],
        #debug_wrapper(xmlcount_reward_func),
        # debug_wrapper(soft_format_reward_func),
        # debug_wrapper(strict_format_reward_func),
        # debug_wrapper(int_reward_func),
        # debug_wrapper(correctness_reward_func)],
    args=training_args,
    train_dataset=dataset,
    peft_config=peft_config
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [39]:
trainer.train()

easy_reward_func: [0.0, 1.0, 1.0, 1.0]


Step,Training Loss
1,-0.000000
2,-0.000000
3,0.000000


easy_reward_func: [0.0, 0.0, 1.0, 0.0]
easy_reward_func: [1.0, 1.0, 1.0, 1.0]
easy_reward_func: [1.0, 1.0, 1.0, 1.0]


KeyboardInterrupt: 